In [1]:
from rosemary import jpt_parse_args, jpt_setup, jpt_in_notebook; jpt_setup()

if jpt_in_notebook():
    import os
    os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3,4,5'
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    

/gpfs/u/scratch/PTFM/PTFMqngp/github/rosemary/src/rosemary/__init__.py:25: UserWarning: Install `torch` for functionalities dependent on torch
  warn(f'Install `torch` for functionalities dependent on torch')


In [2]:
import os
import sys
import numpy as np
import time

import pickle
from tqdm import tqdm 

import torch
import transformers

from sklearn.cluster import KMeans

In [4]:
save_dir = '/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts'
save_path = os.path.join(save_dir, 'note_explore_dpp_llama-7b_flan_v2_outputs.pkl')

with open(save_path, 'rb') as f:
    d = pickle.load(f)
        
# some entries are nan.
d['log_probs'] = np.nan_to_num(d['log_probs'], nan=np.nanmean(d['log_probs']))
text_embeddings = d['text_embeddings']
log_probs = d['log_probs'].squeeze()


In [ ]:
X = text_embeddings[:100,:]

In [14]:
n_clusters_list = [100, 1000, 10000]

for n_clusters in n_clusters_list:

    s = time.time()
    kmeans = KMeans(n_clusters=n_clusters, random_state=0, n_init='auto', verbose=True)
    kmeans.fit(X)

    cluster_labels = kmeans.labels_
    cluster_centers = kmeans.cluster_centers_
    D = np.linalg.norm(X - cluster_centers[cluster_labels], axis=1)
    e = time.time()

    print(f'kmeans took {e-s:.2f} seconds')


    ## sort by increasing/decreasing
    save_path = os.path.join(
        save_dir, f'note_explore_dpp_llama-7b_flan_v2_subsets_kmeansl2_{str(n_clusters)}_incr.pkl')
    inds = np.argsort(D).tolist()
    with open(save_path, 'wb') as f:
        pickle.dump({'K': inds, 'D': D[inds].tolist()}, f, protocol=pickle.HIGHEST_PROTOCOL)

    save_path = os.path.join(
        save_dir, f'note_explore_dpp_llama-7b_flan_v2_subsets_kmeansl2_{str(n_clusters)}_decr.pkl')
    inds = inds[::-1]
    with open(save_path, 'wb') as f:
        pickle.dump({'K': inds, 'D': D[inds].tolist()}, f, protocol=pickle.HIGHEST_PROTOCOL)
        
        
    break


Initialization complete
Iteration 0, inertia 56255617.589423.
Iteration 1, inertia 37275771.54637828.
Iteration 2, inertia 36509988.932683475.
Iteration 3, inertia 36177574.20281856.
Iteration 4, inertia 35989972.61479252.
Iteration 5, inertia 35889952.801850006.
Iteration 6, inertia 35828864.571451455.
Iteration 7, inertia 35790784.343620695.
Iteration 8, inertia 35765308.26719252.
Iteration 9, inertia 35744755.30007022.
Iteration 10, inertia 35728275.02039241.
Iteration 11, inertia 35715301.365089595.
Iteration 12, inertia 35704267.57948327.
Iteration 13, inertia 35694493.38640943.
Iteration 14, inertia 35685382.663028315.
Iteration 15, inertia 35676312.64477272.
Iteration 16, inertia 35668267.8442846.
Iteration 17, inertia 35661137.9111049.
Iteration 18, inertia 35656236.70841251.
Iteration 19, inertia 35652452.80427002.
Iteration 20, inertia 35649103.2499965.
Iteration 21, inertia 35646432.740198314.
Iteration 22, inertia 35643845.559256636.
Iteration 23, inertia 35641054.05398145.

In [54]:

## sort by increasing/decreasing
save_path = os.path.join(
    save_dir, f'note_explore_dpp_llama-7b_flan_v2_subsets_kmeansl2_10_decr.pkl')
with open(save_path, 'rb') as f:
    inds = pickle.load(f)


In [55]:
inds

{'K': [62,
  99,
  41,
  5,
  53,
  29,
  58,
  31,
  86,
  44,
  2,
  51,
  11,
  89,
  38,
  28,
  25,
  12,
  64,
  26,
  1,
  19,
  52,
  8,
  94,
  20,
  83,
  22,
  32,
  27,
  3,
  90,
  45,
  15,
  33,
  46,
  34,
  18,
  80,
  60,
  16,
  23,
  48,
  13,
  96,
  59,
  63,
  40,
  42,
  84,
  77,
  37,
  57,
  61,
  95,
  7,
  85,
  4,
  49,
  30,
  56,
  87,
  43,
  70,
  79,
  97,
  17,
  0,
  88,
  76,
  69,
  91,
  73,
  10,
  78,
  9,
  74,
  65,
  47,
  93,
  72,
  68,
  6,
  92,
  39,
  24,
  14,
  35,
  55,
  81,
  36,
  98,
  75,
  50,
  82,
  21,
  54,
  67,
  71,
  66],
 'D': [49.31555012754102,
  37.12252813820927,
  35.91789732898662,
  34.87003155740098,
  34.765913267161736,
  34.65952086729119,
  33.511454173098564,
  33.33051868600844,
  31.72051043268683,
  31.30296573206358,
  30.931648829719137,
  30.928359035277587,
  30.86583999627714,
  30.178267701482493,
  29.99548258166956,
  29.980336196417944,
  29.92558704005774,
  29.905870788932017,
  29.891103723

(100000,)

array([ True,  True,  True, ...,  True,  True,  True])